In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd

import sys
sys.path.append('functions/')
from read_file import read_file
from filebrowser import FileBrowser

%matplotlib notebook

## File browser:

In [2]:
from filebrowser import FileBrowser
File = FileBrowser()
display(File.widget())

path = File.path

In [25]:
def get_SNR(f, window = 11, plot = False):
    """awdawd"""
    from scipy.signal import savgol_filter
    
    def process(f):
        
        def get_Baseline(C):
            """Returns baseline of transient ёC"""
            l = len(C)
            c1, c2, c3 = C[0], C[int(l/2)-1], C[l-1]
            return (c1*c3 - c2**2)/(c1+c3 - 2*c2)

        F = np.asarray(f)
        F = F/F[0]
        if F[0] > F[-1]:
            F = F - min(F)
        else:
            F = F - max(F)
        F = np.abs(F)
        F = F + np.average(F)*2
        F = F - get_Baseline(F)
        return np.asarray(F)
    
    f = process(f)
    f_s = savgol_filter(f, window, 2)
    
    
    SNR = f_s/np.abs(f-f_s)
    SNR_ave = np.average(SNR[20:], weights = (1/SNR[20:])**1)
    if plot:
        fig = plt.figure()
        ay  = fig.add_subplot(211)
        ay.plot(f,   'k-' , label = 'Original', lw = 2)
        ay.plot(f_s, 'r-' , label = 'Smoothed', lw = 1)
        ay.set_ylabel('Transient (arb. units)')
        ay.set_xlabel('Time (arb. units)')
        ay.legend()
        
        ax  = fig.add_subplot(212)
        ax.set_yscale('log')
        ax.plot(SNR, 'x')
        ax.set_ylabel('SNR')
        ax.set_xlabel('Time (arb. units)')
        ax.axhline(SNR_ave, c = 'r', label = 'Average')
        ax.legend()
        
        ax.axhline(y = 500, color = 'k', alpha = 0.5) # emission rate resolution e1/e2 = 2
        ax.annotate(r'$\delta = 2$',   xy = (1, 500), color = 'k',   alpha = 0.5, va = 'bottom')
        ax.axhline(y = 15000, color = 'k', alpha = 0.5) # emission rate resolution e1/e2 = 1.6
        ax.annotate(r'$\delta = 1.6$', xy = (1, 15000), color = 'k', alpha = 0.5, va = 'bottom')
    
        plt.tight_layout()
        #plt.savefig('SNR_one.svg')
        
    return SNR_ave

In [26]:
def draw_SNR(df, window = 31):
    """awdawd"""
    s = []
    F = 0
    count = 0
    
    for i, col in df.iteritems():
        F = F + col
        count = count + 1
        if count >= stop:
            break
        s.append(get_SNR(F/count, window))
    return np.asarray(s)

In [27]:
def average_spectra(path, dT = 0.3, save = False):
    '''Function to average captured Laplace DLTS spectra
    path – path to file to precess
    save – True for saving final data
     
    '''
    
    from scipy.optimize import curve_fit
    
    def sqrt(x, a):
        return a*np.sqrt(x)

    def cluster_T(T, dT):
        """Cluster Temperature array
        T – initial array
        
        T_av – clustered data
        
        """
        T_cl = [T[0]]
        for i, _T in enumerate(T):
            if i != 0:
                if np.abs(T[i] - T[i-1]) >= dT:
                    T_cl.append(T[i])

        T_cl = np.array(T_cl)
        
        T_av = []
        for i, _Tc in enumerate(T_cl):
            temp  = 0
            count = 0
            for j, _T in enumerate(T):
                if np.abs(_Tc - _T) < dT:
                    temp  = temp  + _T
                    count = count + 1
            T_av.append(temp/count)
                    
        return np.array(T_av)
    
    def average_columns(df, T_av, dT):
        '''Averaging tra'''
        
        data = []
        for i, _T in enumerate(T_av):
            temp  = 0
            count = 0
            for i, col in df.iteritems():
                if np.abs(_T - i) < dT:
                    temp  = temp  + col
                    count = count + 1
            data.append(temp/count)
            
        data = np.array(data)
        return pd.DataFrame(data.T, columns = T_av)
    
    def save_spectra(df, save):
        T = np.asmatrix(df.columns)
        C = np.asmatrix(df)
        
        if save:
            
            #data = np.append(T.T, C.T, axis = 1)
            Z = np.zeros(T.T.shape)
            #print(Z.shape)
            data = np.append(T.T,    Z, axis = 1)
            data = np.append(data, C.T, axis = 1)
            data = np.append(data,   Z, axis = 1)
            data = np.append(data,   Z, axis = 1)
            #print(data)
            np.savetxt('data/averaged/averaged.DLTS', data, fmt='%.28e', delimiter='\t')
        
        return df2   
    
    
    t, C, T = read_file(path)
    print('Tolal transients %.0f'%len(C))
    
    
    df = pd.DataFrame(C.T, columns = T)
    df = df.sort_index(axis=1)

    # Clustering temperature
    T_av = cluster_T(df.columns, dT)
    
    # Averaging transienst
    df2 = average_columns(df, T_av, dT)
    
    # Saving averaged data
    save_spectra(df2, save)
    
    #display(df)
    #display(df2)
    fig = plt.figure(figsize = (6,12))
    ax1 = fig.add_subplot(211)
    ax1.set_xscale('log')
    
    ax1.plot(df, c = 'k', lw = 0.2)
    
    color = cm.gnuplot2(np.linspace(0, 0.9, len(df2.columns)))
    count = 0
    for i, col in df2.iteritems():
        ax1.plot(col, lw = 2, c = color[count], label = '%.2f K'%i)
        count = count + 1
    ax1.legend(loc = 'upper right')
    ax1.set_ylabel('Transient amplitude $C(t)/C_0$ (arb. units)')
    ax1.set_xlabel('Time $t$ (arb. units)')
    
    ax2 = fig.add_subplot(212) 
    SNR = draw_SNR(df, window = 31)
    n = np.linspace(1, len(SNR), len(SNR))
    ax2.set_yscale('log')
    #ax2.set_xscale('log')
    count = 0
    for i, _SNR in enumerate(SNR):
        ax2.scatter(i+1, _SNR, c = color[count], label = 'SNR')
        count = count + 1
        
    ax2.set_ylabel('SNR')
    ax2.set_xlabel('Transient number')
    
    ax2.axhline(y = 500, color = 'k', alpha = 0.5) # emission rate resolution e1/e2 = 2
    ax2.annotate(r'$\delta = 2$',   xy = (1, 500), color = 'k',   alpha = 0.5, va = 'bottom')
    ax2.axhline(y = 15000, color = 'k', alpha = 0.5) # emission rate resolution e1/e2 = 1.6
    ax2.annotate(r'$\delta = 1.6$', xy = (1, 15000), color = 'k', alpha = 0.5, va = 'bottom')
    
    popt, pcov = curve_fit(sqrt, n, SNR, p0 = 100)
    #ax2.plot(n, sqrt(n, *popt), label = 'fit')
    plt.tight_layout()
    #plt.savefig('SNR.svg')
    initial = df.iloc[:,1]
    print('SNR for initial transiens is %.1e'%get_SNR(initial, window = 31, plot = True))

Tolal transients 645


<IPython.core.display.Javascript object>


*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
/Users/antonvasilev/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/antonvasilev/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations


<IPython.core.display.Javascript object>


SNR for initial transiens is 5.5e+00


In [30]:
stop = 2 # 2 for one transient to be processed

average_spectra(File.path, dT = 0.3, save = True)

* SNR $\propto$ round
* SNR $\propto$ $\sqrt{N}$
* SNR $\propto$ ???